<img style="float: left;" src="D2Klogo.png" width="20%"> 

<center>

# Welcome to COMP 680 #
### Statistics for Computing and Data Science ###

### Week 9 Bayesian Inference ###

In [1]:
### standard imports
import numpy as np
import pandas as pd

import scipy.stats
import scipy.optimize as opt
#import statsmodels.api as sm

%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams["patch.force_edgecolor"] = True
import seaborn as sns

import random
random.seed(2023)

## Binomial Beta Model ##

In [ ]:
# First we generate our data from Bernoulli distribution with success probability p (the chance of getting Head)
theta = 0.5 # theta is the head prob
n = 100 # sample size
# generate n bernoulli trial with success prob theta
data = np.random.binomial(1, theta, n) # generate n bernoulli trial 

plt.hist(data)
plt.xlabel('data')
plt.ylabel('count')
plt.title('Data from Binomial('+str(n) + ', '+ str(theta)+')');

In [ ]:
# sample mean x_n
sum_x = sum(data) # number of heads
theta_mle = np.mean(data) # sample mean as MLE
theta_mle

In [ ]:
# Plot our likelihood function as a function of parameter theta
# generate a grid of theta between 0 and 1
theta_plot = np.arange(0.01, 1.00, 0.01)
# calculate the likelihood function on the grid
l_theta = theta_plot**(sum_x)*(1 - theta_plot)**(n-sum_x)
# calculate the loglikelihood function on the grid
ll_theta = sum_x * np.log(theta_plot) +  (n-sum_x) * np.log(1 - theta_plot)

# plot the likelihood 
plt.plot(theta_plot, l_theta, label='likelihood')
plt.plot([theta_mle, theta_mle], [min(l_theta), max(l_theta)], label='theta_mle')
plt.xlabel('theta')
plt.ylabel('likelihood')
plt.legend();

In [ ]:
# plot the log likelihood 
plt.plot(theta_plot, ll_theta, label='log_likelihood')
plt.plot([theta_mle, theta_mle], [min(ll_theta), max(ll_theta)], label='theta_mle')
plt.xlabel('theta')
plt.ylabel('log_likelihood')
plt.legend();

In [ ]:
# We choose our prior distribution for the unknown parameter head_prob to be a Beta distribution
# Beta distribution has two parameters a and b 
# with mean = a/(a+b) and variance = ab/[(a+b)**2*(a+b+1)]
# Plot a histogram to see the shape of prior with respect to different choices of prior parameters
# try (a, b) = (10, 10), (1, 1), (1, 10), (10, 1), (0.5, 0.5) to see the difference
prior_a = 1
prior_b = 10
prior_mean = prior_a / (prior_a + prior_b)
prior = np.random.beta(a = prior_a, b = prior_b, size = 10000)
P_df = pd.DataFrame({'prior': prior})

# seaborn displot will plot histogram with a kde
# kde: kernel density estimate
sns.distplot(P_df)
plt.xlabel('theta')
plt.ylabel('density')
plt.title('prior distribution Beta('+ str(prior_a) + ', '+str(prior_b)+')');

In [ ]:
# Our posterior distribution is still a Beta distribution due to conjugacy
# The two parameters for Beta are updated by observed data (likelihood)
post_a = prior_a + sum_x
post_b = prior_b + (n - sum_x)
post_mean = post_a / (post_a + post_b)
post = np.random.beta(a = post_a, b = post_b, size = 10000)
P_df['posterior'] = post

sns.distplot(P_df['posterior'])
plt.xlabel('theta')
plt.ylabel('density')
plt.title('posterior distribution Beta('+ str(post_a) + ', '+str(post_b)+')');

In [ ]:
# Let's plot prior and posterior distributions together
# sns.distplot(P_df);

print('Likelihood model: Binomial('+str(n) + ', '+ str(theta)+')')
print('Prior: Beta('+ str(prior_a) + ', ' + str(prior_b)+')')
print('Posterior: Beta('+ str(post_a) + ', '+ str(post_b)+')')
print('MLE: ', theta_mle)
print('Prior mean: %.2f' %prior_mean)
print('Posterior mean: %.2f' %post_mean)

sns.distplot(P_df['prior'], label='prior')
sns.distplot(P_df['posterior'], label='post')

plt.xlabel('theta')
plt.ylabel('density')
plt.title('Binomial Beta Model')
plt.legend();

## Poisson Gamma Model


### Likelihood: Poisson Distribution
Recall that the Poisson distribution is given by:

$$\mathbb{P}(X = x_i | \lambda) = \frac{\lambda^{x_i} e^{-\lambda}} {x_i !}$$

where $\lambda > 0$ is the Poisson parameter, and it is easy to verify that:
$$\mathbb{E}[X] = \mathbb{V}(X) = \lambda$$

### Prior: Gamma Distribution
Recall that the density function for the gamma distribution is

$$ f(\lambda |a, b) = \frac{b^a}{\Gamma(a)} \lambda^a e^{-b\lambda}$$ 

where $a>0$ is the shape parameter and $b>0$ is the rate parameter.

The expected value and variance of the gamma distribution is
$$\mathbb{E}[\lambda] = \frac{a}{b} \quad \mathbb{V}(\lambda) = \frac{a}{b^2}$$

In [ ]:
# put everything in a function
def simu_poisson_gamma(po_lam, ga_a, ga_b, n=100, plot_n=1000):
    '''
    n: sample size of simulated data
    po_lam: parameter lambda in poisson likelihood
    ga_a: parameter a in gamma prior
    ga_b: parameter b in gamma prior, 
          notice in scipy.stats.gamma scale = 1/b in
    plot_n: grid number to plot for prior and post density
    '''
    # generate data with sample size n from likelihood model
    data = np.random.poisson(lam=po_lam, size=n)
    # calculate sample mean
    sample_mean = np.mean(data)
    
    # plot data
    data_plot = plt.hist(data, density=True, alpha=0.5)
    plt.xlabel('data')
    plt.ylabel('count')
    plt.title(str(n)+' data from Poisson('+ str(po_lam)+')')
    plt.axvline(sample_mean, color='orange', label='sample mean', linewidth=3)
    plt.legend()
    plt.show();
    
    # calculate prior
    # prior parameters are passed by arguments
    prior = f'Prior Gamma(a={ga_a}, b={ga_b})'
    # prior mean
    prior_mean = ga_a / ga_b
    # create a grid for plot lambda and its density
    lam_plot = np.linspace(start=0, stop=max(data)+1, num=plot_n)
    # prior pdf using prior parameters
    prior_pdf = scipy.stats.gamma.pdf(lam_plot, a=ga_a, scale=1/ga_b)
    
    # calculate log likelihood
    likelihood = f'Likelihood Poisson(lambda={po_lam})'
    lam_ll = lam_plot + 1e-6 # add a small number to avoid log(0)
    # log likelihood function up to a constant
    ll = sum(data)*np.log(lam_ll) - n*lam_ll
    # rescaled for plotting with prior and posterior
    rescale_ll = (ll - np.min(ll)) / (np.max(ll) - np.min(ll))
    
    # calculate posterior
    # update posterior parameters
    po_a = ga_a + sum(data)
    po_b = ga_b + n
    post = f'Post Gamma(a={po_a}, b={po_b})'
    # post mean
    post_mean = po_a / po_b
    # post pdf using post parameters
    post_pdf = scipy.stats.gamma.pdf(lam_plot, a=po_a, scale=1/po_b)
    
    # likelihood, prior and post plot in one
    # put everything in a dataframe for easy plotting
    # 4 columns: lambda grid, rescaled logll, prior density, post density
    prior_post_df = pd.DataFrame({'lambda':lam_plot, 'loglikelihood':rescale_ll, 'prior':prior_pdf, 'post':post_pdf})
    # plot 3 lines
    plot = prior_post_df.plot(x='lambda', linewidth=3)
    plt.title('Poisson Gamma Model Plot')
    plt.xlabel('Poisson parameter lambda')
    plt.ylabel('density')
    plt.show();
    
    # print results
    print(likelihood)
    print('Sample size of data: %.0f' %n)
    print(prior)
    print(post)
    print('MLE of lambda is sample mean: ', sample_mean)
    print('Prior mean: %.2f' %prior_mean)
    print('Posterior mean: %.2f' %post_mean)
    

In [ ]:
# call the function
po_lam = 2
ga_a = 1
ga_b = 1
n = 100
simu_poisson_gamma(po_lam, ga_a, ga_b, n)

## Posterior Inference using Markov Chain Monte Carlo (MCMC) Approach
In the last two examples the posterior distributions were analytically avaliable due to simple models (only one unknown parameter and conjugate priors). However, in paractice this is not usually the case and therefore, via Bayes Theorem, we would only know the posterior distribution up to a constant. This motivates the idea of using Monte Carlo simulation methods. How can we sample from a distribution that we do not know? The anwser is Gibbs sampling or Metropolis–Hastings algorithm.

Python has a library [PyMC3](https://docs.pymc.io/), a friendly modelling API to implement Bayesian models. Feel free to explore this tutorial [Introduction to Bayesian Modeling with PyMC3](https://juanitorduz.github.io/intro_pymc3/).